# Amazon rainforests

### MobileNetV2


In [26]:
import pandas as pd 
import numpy as np 
np.random.seed(0)
import os.path
# import os
import gc

import tensorflow.keras as k
from tensorflow.keras import backend as K

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D

from tensorflow.keras.applications import  MobileNetV2
from tensorflow.keras.models import Model

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import fbeta_score
from tensorflow.keras.optimizers import Adam

#import cv2
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [27]:
df_train = pd.read_csv('dane/train_v2.csv')

Przygotowujemy odpowiedzi (`lables`).

In [28]:
all_labels = df_train['tags'].map(lambda x: x.split(' ')).values
labels = list(set([y for x in all_labels for y in x]))

print( len(labels), labels )

17 ['selective_logging', 'artisinal_mine', 'blooming', 'primary', 'habitation', 'conventional_mine', 'slash_burn', 'agriculture', 'cloudy', 'water', 'clear', 'bare_ground', 'blow_down', 'partly_cloudy', 'road', 'haze', 'cultivation']


In [29]:
def read_data(df_train, labels, resize=(32, 32)):
    X_train = []
    y_train = []

    label_map = {l: i for i, l in enumerate(labels)}
    inv_label_map = {i: l for l, i in label_map.items()}

    for f, tags in tqdm(df_train.values, miniters=1000):
        if False == os.path.exists('../input/amazon/train-jpg/{}.jpg'.format(f)): continue

        img = cv2.imread('../input/amazon/train-jpg/{}.jpg'.format(f))
        targets = np.zeros(17)
        for t in tags.split(' '):
            targets[label_map[t]] = 1 

        X_train.append(cv2.resize(img, resize))
        y_train.append(targets)
        
    y_train = np.array(y_train, np.uint8)
    X_train = np.array(X_train, np.float16) / 255.
    
    split = 5000#35000
        
    return X_train[:split], X_train[split:], y_train[:split], y_train[split:]

In [30]:
def draw_learning_curve(history, key='acc', ylim=(0.8, 1.01)):
    plt.figure(figsize=(12,6))
    plt.plot(history.history[key])
    plt.plot(history.history['val_' + key])
    plt.title('Learning Curve')
    plt.ylabel(key.title())
    plt.xlabel('Epoch')
    plt.ylim(ylim)
    plt.legend(['train', 'test'], loc='best')
    plt.show()

In [31]:
def fbeta_score_K(y_true, y_pred):
    beta_squared = 4

    tp = K.sum(y_true * y_pred) + K.epsilon()
    fp = K.sum(y_pred) - tp
    fn = K.sum(y_true) - tp

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    result = (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

    return result

In [33]:
del X_train, X_test, y_train, y_test
gc.collect()

80

In [34]:
X_train, X_test, y_train, y_test = read_data(df_train, labels, resize=(224, 224))

100%|██████████| 40479/40479 [00:00<00:00, 208997.27it/s]


In [35]:
MobileNetV2

<function tensorflow.python.keras.applications.mobilenet_v2.MobileNetV2(input_shape=None, alpha=1.0, include_top=True, weights='imagenet', input_tensor=None, pooling=None, classes=1000, classifier_activation='softmax', **kwargs)>

In [36]:
weights = 'mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5'
base_model = MobileNetV2(weights=weights, include_top=False, input_shape=(224, 224,3))
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [24]:
for layer in base_model.layers:
    layer.trainable = False

x = base_model.layers[-6].output
output = Dense(units=17, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

optimizer = Adam(0.003, decay=0.0005)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy', fbeta_score_K])

model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [44]:
callbacks = [
    ModelCheckpoint('MobileNetV2.hdf5', monitor='val_loss', save_best_only=False, verbose=2, save_weights_only=False),
    EarlyStopping(monitor='val_loss', patience=5, verbose=0)
]

history = model.fit(X_train, y_train,
          batch_size=64,
          epochs=3,
          verbose=2,
          callbacks = callbacks,
          validation_data=(X_test, y_test))

Epoch 1/3


UnboundLocalError: local variable 'logs' referenced before assignment